In [1]:
!pip3 install riotwatcher

     |████████████████████████████████| 56 kB 3.8 MB/s 


In [28]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

#variáveis globais
api_key = 'RGAPI-9ac79a8b-9632-43b5-9bc6-2fbc4db2a4f2'
watcher = LolWatcher(api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'Doublelift')
print(me)

{'id': 'hMDkuC6Kii4MqadrK0pAtvk1fbux1hV3er7w8hRSMQt2qFc', 'accountId': 'yfqEZoEOehaMEaLrO0dSV94kwy36U35NRjyrE64AXdEMng', 'puuid': '4WBvjGr5_b4gxbd95H_ixsn23312PH5Q-kDSarEJjzVcPjEIMf3925IqhbyyduQMRMMDKAva61Yaqw', 'name': 'Doublelift', 'profileIconId': 4568, 'revisionDate': 1636147167000, 'summonerLevel': 274}


In [15]:
# Retorna o status do rank de Doublelift
my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

[{'leagueId': '2bd81059-a0ce-33d4-9fb0-86977c851f64', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'CHALLENGER', 'rank': 'I', 'summonerId': 'hMDkuC6Kii4MqadrK0pAtvk1fbux1hV3er7w8hRSMQt2qFc', 'summonerName': 'Doublelift', 'leaguePoints': 937, 'wins': 346, 'losses': 301, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


In [72]:
# Acessando dados de partida
region = "americas"
my_matches = watcher.match.matchlist_by_puuid(region, me['puuid'])
last_match = my_matches[-1]
last_match

match_detail = watcher.match.by_id(region, last_match)

In [53]:
participants = []

for row in match_detail['info']['participants']:
  participants_row = {}
  participants_row['champion'] = row['championId']
  participants_row['summoner1Id'] = row['summoner1Id']
  participants_row['summoner2Id'] = row['summoner2Id']
  participants_row['win'] = row['win']
  participants_row['kills'] = row['kills']
  participants_row['deaths'] = row['deaths']
  participants_row['assists'] = row['assists']
  participants_row['totalDamageDealt'] = row['totalDamageDealt']
  participants_row['goldEarned'] = row['goldEarned']
  participants_row['champLevel'] = row['champLevel']
  participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
  participants_row['item0'] = row['item0']
  participants_row['item1'] = row['item1']
  participants.append(participants_row)

df=pd.DataFrame(participants)
df

,champion,summoner1Id,summoner2Id,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,39,12,4,False,2,8,4,133491,11028,14,208,3091,3044
1,13,12,4,False,4,10,5,199470,12404,16,236,1057,6656
2,163,4,11,False,2,8,6,140787,9753,15,34,3157,3916
3,21,4,7,False,10,3,4,210434,16255,16,284,3140,6692
4,235,3,4,False,3,7,11,30664,9124,13,26,2055,6632
5,85,4,12,True,12,3,5,207407,15186,18,237,0,3152
6,141,4,11,True,10,5,10,272752,15230,17,56,6630,3142
7,777,4,12,True,10,1,7,228077,16353,18,256,6333,3006
8,115,12,4,True,4,7,9,173438,12982,16,217,1058,6655
9,497,14,4,True,0,5,11,16068,7696,13,43,2421,3860


In [71]:
# Acessando Data Dragon

# League lastest version
latest = watcher.data_dragon.versions_for_region(my_region)['n']['champion']
static_champ_list = watcher.data_dragon.champions(latest, False, 'en_US')

# Champ static list data to dict for looking up
champ_dict = {}
for key in static_champ_list['data']:
  row = static_champ_list['data'][key]
  champ_dict[row['key']] = row['id']

for row in participants:
  print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
  row['championName'] = champ_dict[str(row['champion'])]

df = pd.DataFrame(participants)
df

# Outros tipos de informações como spells, runas, e itens estão disponíveis
# em DataDragon
# para mais informações, ler a doc da RIOT_API

39 Irelia
13 Ryze
163 Taliyah
21 MissFortune
235 Senna
85 Kennen
141 Kayn
777 Yone
115 Ziggs
497 Rakan


,champion,summoner1Id,summoner2Id,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,championName
0,39,12,4,False,2,8,4,133491,11028,14,208,3091,3044,Irelia
1,13,12,4,False,4,10,5,199470,12404,16,236,1057,6656,Ryze
2,163,4,11,False,2,8,6,140787,9753,15,34,3157,3916,Taliyah
3,21,4,7,False,10,3,4,210434,16255,16,284,3140,6692,MissFortune
4,235,3,4,False,3,7,11,30664,9124,13,26,2055,6632,Senna
5,85,4,12,True,12,3,5,207407,15186,18,237,0,3152,Kennen
6,141,4,11,True,10,5,10,272752,15230,17,56,6630,3142,Kayn
7,777,4,12,True,10,1,7,228077,16353,18,256,6333,3006,Yone
8,115,12,4,True,4,7,9,173438,12982,16,217,1058,6655,Ziggs
9,497,14,4,True,0,5,11,16068,7696,13,43,2421,3860,Rakan
